In [ ]:
import numpy as np
import yt
import glob
import os
import matplotlib.pyplot as plt
from mpi4py import MPI

In [ ]:
Problem_Type =["supernova", "DAVID_SUPERNOVA"]
glob_pattern = ["/home/david/RUNS/run_data//id0/*vtk", "/home/david/FLASH_RUNS//*plt_cnt*"]

In [ ]:
for j in range (len(Problem_Type)):
    os.system("rm " + glob_pattern[j].split("//")[0]+"/"+Problem_Type[j]+"/*forced*")
    files = np.sort(glob.glob(glob_pattern[j].replace("//", "/"+Problem_Type[j]+"/")))
    time = np.zeros(len(files))
    radii = np.zeros(len(files))
    for i in range (len(files)):
        print(files[i])
        ds = yt.load(files[i])
        time[i] = ds.current_time
        ad = ds.all_data()
        pressure = ad["pressure"]
        density = ad["density"]
        if i == 0:
            indices = pressure == pressure.max()
        else:
            indices = density == density.max()
        radius = np.sqrt(ad["x"][indices]**2 + ad["y"][indices]**2 + ad["z"][indices]**2)
        radii[i] = np.max(radius)
        print(np.min(radius))
        print(np.max(radius))

In [ ]:
comm = MPI.COMM_WORLD
rank = comm.Get_rank() 

for j in range (len(Problem_Type)):
    os.system("rm " + glob_pattern[j].split("//")[0]+"/"+Problem_Type[j]+"/*forced*")
    files = np.sort(glob.glob(glob_pattern[j].replace("//", "/"+Problem_Type[j]+"/")))
    time = np.zeros(len(files))
    radii = np.zeros(len(files))
    files2 = []
    for k in range (len(files)):
        if k % 4 == rank:
            files2.append(files[k])

    print(rank, files2)
    break
    for i in range (len(files)):
        print(files[i])
        ds = yt.load(files[i])
        time[i] = ds.current_time
        ad = ds.all_data()
        pressure = ad["pressure"]
        density = ad["density"]
        if i == 0:
            indices = pressure == pressure.max()
        else:
            indices = density == density.max()
        radius = np.sqrt(ad["x"][indices]**2 + ad["y"][indices]**2 + ad["z"][indices]**2)
        radii[i] = np.max(radius)
        print(np.min(radius))
        print(np.max(radius))


In [ ]:
comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()

DEBUG = True

for j in range (len(Problem_Type)):
    os.system("rm " + glob_pattern[j].split("//")[0]+"/"+Problem_Type[j]+"/*forced*")
    files = np.sort(glob.glob(glob_pattern[j].replace("//", "/"+Problem_Type[j]+"/")))
    if DEBUG:
        files = files[:12]
    files2 = []
    for k in range (len(files)):
        if k % size == rank:
            files2.append(files[k])

    time = np.zeros(len(files2))
    radii = np.zeros(len(files2))
    #print("\n\n\n", rank, files2, "\n\n\n")
    for i in range (len(files2)):
        print(files2[i])
        ds = yt.load(files2[i])
        time[i] = ds.current_time
        ad = ds.all_data()
        pressure = ad["pressure"]
        density = ad["density"]
        if i == 0:
            indices = pressure == pressure.max()
        else:
            indices = density == density.max()
        radius = np.sqrt(ad["x"][indices]**2 + ad["y"][indices]**2 + ad["z"][indices]**2)
        radii[i] = np.max(radius)
    #      print(np.min(radius))
    #      print(np.max(radius))

    radii = comm.gather(radii, root=0)
    time = comm.gather(time, root=0)
    if rank != 0:
        exit()
    radii = np.concatenate(
    #radii2 = np.array([])
    #time2 = np.array([])
    #for i in range (len(radii)):
    #    radii2 = np.append(radii
    #    time2 += time[i]
    print(rank, type(radii), type(time), time)
    print(radii2, time2)
    exit()
    plt.plot(time, radii)

In [ ]:
plt.plot(time, radii)
plt.xlabel("Time [code time]")
plt.ylabel("Radii [code length]") 
plt.show()